In [ ]:
import tensorflow as tf
from tensorflow.python.client import device_lib
print(tf.__version__)
# Creates a graph.
a = tf.constant([1.0, 2.0, 3.0, 4.0, 5.0, 6.0], shape=[2, 3], name='a')
b = tf.constant([1.0, 2.0, 3.0, 4.0, 5.0, 6.0], shape=[3, 2], name='b')
c = tf.matmul(a, b)
# Creates a session with log_device_placement set to True.
sess = tf.Session(config=tf.ConfigProto(log_device_placement=True))
# Runs the op.
print(sess.run(c))
print(tf.test.is_gpu_available())
print(device_lib.list_local_devices())


In [9]:
# Import Tensorflow and Numpy
import tensorflow as tf
import numpy as np

# ======================
# Define the Graph
# ======================

# Create Placeholders For X And Y (for feeding in data)
X = tf.placeholder("float",[10, 10],name="X") # Our input is 10x10
Y = tf.placeholder("float", [10, 1],name="Y") # Our output is 10x1

# Create a Trainable Variable, "W", our weights for the linear transformation
initial_W = np.zeros((10,1))
W = tf.Variable(initial_W, name="W", dtype="float32")

# Define Your Loss Function
Loss = tf.pow(tf.add(Y,-tf.matmul(X,W)),2,name="Loss")

with tf.Session() as sess: # set up the session
    sess.run(tf.initialize_all_variables())
    Model_Loss = sess.run(
                Loss, # the first argument is the name of the Tensorflow variabl you want to return
                { # the second argument is the data for the placeholders
                  X: np.random.rand(10,10),
                  Y: np.random.rand(10).reshape(-1,1)
                })
    print(Model_Loss)

Instructions for updating:
Use `tf.global_variables_initializer` instead.
[[0.00612033]
 [0.11630105]
 [0.01456722]
 [0.03699512]
 [0.09238198]
 [0.46243307]
 [0.12236829]
 [0.23398192]
 [0.01862067]
 [0.05418464]]


In [20]:
#!/usr/bin/env python
# -*- encoding: utf-8 -*-
"""
@File    :   RNN.py    
@Contact :   liumingshanneo@163.com

@Modify Time      @Author    @Version
------------      -------    --------
2020/1/31 9:11   msliu      1.0      

@Description
------------
None
"""
import time

"""
This is a sample RNN code.
Please make some changes instead of submitting the original code.
If you have some questions about the code or data,
please contact Y.Q. Deng(yqdeng@cs.hku.hk) or M.M. Kuang(kuangmeng@hku.hk)
"""

import numpy as np
import tensorflow as tf

# Prepare Data(Training and Testing)
filename = "data.txt"
a_list = []
b_list = []
c_list = []


def str_2_list(data_list):
    ret_list = []
    for i in range(len(data_list)):
        tmp_list = data_list[i].strip().split(" ")
        tmp_ret_list = [int(tmp_list[7][0]), int(tmp_list[6]), int(tmp_list[5]), int(tmp_list[4]), int(tmp_list[3]),
                        int(tmp_list[2]), int(tmp_list[1]), int(tmp_list[0][1])]  # changed
        ret_list.append(tmp_ret_list)
    return ret_list


with open(filename, "r") as file:
    filein = file.read().splitlines()
    for item in filein:
        tmp_list = item.strip().split(",")
        a_list.append(tmp_list[0])
        b_list.append(tmp_list[1])
        c_list.append(tmp_list[2])
a_list = str_2_list(a_list)
b_list = str_2_list(b_list)
c_list = str_2_list(c_list)

# Define the dataflow graph
time_steps = 8  # time steps which is the same as the length of the bit-string
input_dim = 2  # number of units in the input layer
hidden_dim = 16  # number of units in the hidden layer
output_dim = 1  # number of units in the output layer
binary_dim = 8
largest_number = pow(2, binary_dim)

tf.reset_default_graph()
# input X and target ouput Y
X = tf.placeholder(tf.float32, [None, time_steps, input_dim], name='x')
Y = tf.placeholder(tf.float32, [None, time_steps], name='y')

# define the RNN cell: can be simple cell, LSTM or GRU
cell = tf.contrib.rnn.BasicRNNCell(num_units=hidden_dim, activation=tf.nn.sigmoid)

# values is a tensor of shape [batch_size, time_steps, hidden_dim]
# last_state is a tensor of shape [batch_size, hidden_dim]
values, last_state = tf.nn.dynamic_rnn(cell, X, dtype=tf.float32)
values = tf.reshape(values, [time_steps, hidden_dim])

# put the values from the RNN through fully-connected layer
W = tf.Variable(tf.random_uniform([hidden_dim, output_dim], minval=-1.0, maxval=1.0), name='W')
b = tf.Variable(tf.zeros([1, output_dim]), name='b')
h = tf.nn.sigmoid(tf.matmul(values, W) + b, name='h')

# minimize loss, using ADAM as weight update rule
h_ = tf.reshape(h, [time_steps])
Y_ = tf.reshape(Y, [time_steps])
loss = tf.reduce_mean(-Y_ * tf.log(h_) - (1 - Y_) * tf.log(1 - h_), name='loss')

train_step = tf.train.AdamOptimizer(0.1).minimize(loss)

# Launch the graph
sess = tf.Session()
sess.run(tf.global_variables_initializer())
start = time.time()
with tf.device("/GPU:0"):
    # Number of data for training and testing
    # Please remember the total number is 5000
    num4train = 4000
    num4test = 1000
    epoch = 2
    # train
    for i in range(epoch):
        for j in range(num4train):

            a = np.array(a_list[j], dtype=np.uint8)  # changed
            b = np.array(b_list[j], dtype=np.uint8)  # changed
            c = np.array(c_list[j], dtype=np.uint8)  # changed
            ab = np.c_[a, b]
            x = np.array(ab).reshape([1, binary_dim, 2])
            y = np.array(c).reshape([1, binary_dim])
            if j % 100 == 0:
                print("Epoch: %d Example: %d is running..." % (i, j))
                a = sess.run(loss, feed_dict={X: x, Y: y})
                # print(a)
                # if a < 0.001:
                #     break;
            sess.run(train_step, {X: x, Y: y})

    remain_result = []
    time_result = time.time()-start
    print("10 loops: {:0.2f}ms".format(1000*time_result))
    # print(X.device.endswith("CPU:0"))
    # print(X.device.endswith("GPU:0"))

# Test
# for i in range(num4train + 1, num4train + num4test):
#     a = np.array(a_list[i], dtype=np.uint8)  # changed
#     b = np.array(b_list[i], dtype=np.uint8)  # changed
#     c = np.array(c_list[i], dtype=np.uint8)  # changed
#     ab = np.c_[a, b]
#     x = np.array(ab).reshape([1, binary_dim, 2])
#     y = np.array(c).reshape([1, binary_dim])
#
#     # get predicted value
#     [_probs, _loss] = sess.run([h, loss], {X: x, Y: y})
#     probs = np.array(_probs).reshape([8])
#     prediction = np.array([1 if p >= 0.5 else 0 for p in probs]).reshape([8])
#     # Save the result
#     remain_result.append([prediction, y[0]])
#
#     # calculate error
#     error = np.sum(np.absolute(y - probs))
#
#     # print the prediction, the right y and the error.
#     print("---------------")
#     print(prediction)
#     print(y[0])
#     print(error)
#     print("---------------")
#     print()

sess.close()

Epoch: 0 Example: 0 is running...
Epoch: 0 Example: 100 is running...
Epoch: 0 Example: 200 is running...
Epoch: 0 Example: 300 is running...
Epoch: 0 Example: 400 is running...
Epoch: 0 Example: 500 is running...
Epoch: 0 Example: 600 is running...
Epoch: 0 Example: 700 is running...
Epoch: 0 Example: 800 is running...
Epoch: 0 Example: 900 is running...
Epoch: 0 Example: 1000 is running...
Epoch: 0 Example: 1100 is running...
Epoch: 0 Example: 1200 is running...
Epoch: 0 Example: 1300 is running...
Epoch: 0 Example: 1400 is running...
Epoch: 0 Example: 1500 is running...
Epoch: 0 Example: 1600 is running...
Epoch: 0 Example: 1700 is running...
Epoch: 0 Example: 1800 is running...
Epoch: 0 Example: 1900 is running...
Epoch: 0 Example: 2000 is running...
Epoch: 0 Example: 2100 is running...
Epoch: 0 Example: 2200 is running...
Epoch: 0 Example: 2300 is running...
Epoch: 0 Example: 2400 is running...
Epoch: 0 Example: 2500 is running...
Epoch: 0 Example: 2600 is running...
Epoch: 0 Exam

In [21]:
#!/usr/bin/env python
# -*- encoding: utf-8 -*-
"""
@File    :   RNN.py    
@Contact :   liumingshanneo@163.com

@Modify Time      @Author    @Version
------------      -------    --------
2020/1/31 9:11   msliu      1.0      

@Description
------------
None
"""
import time

"""
This is a sample RNN code.
Please make some changes instead of submitting the original code.
If you have some questions about the code or data,
please contact Y.Q. Deng(yqdeng@cs.hku.hk) or M.M. Kuang(kuangmeng@hku.hk)
"""

import numpy as np
import tensorflow as tf

# Prepare Data(Training and Testing)
filename = "data.txt"
a_list = []
b_list = []
c_list = []


def str_2_list(data_list):
    ret_list = []
    for i in range(len(data_list)):
        tmp_list = data_list[i].strip().split(" ")
        tmp_ret_list = [int(tmp_list[7][0]), int(tmp_list[6]), int(tmp_list[5]), int(tmp_list[4]), int(tmp_list[3]),
                        int(tmp_list[2]), int(tmp_list[1]), int(tmp_list[0][1])]  # changed
        ret_list.append(tmp_ret_list)
    return ret_list


with open(filename, "r") as file:
    filein = file.read().splitlines()
    for item in filein:
        tmp_list = item.strip().split(",")
        a_list.append(tmp_list[0])
        b_list.append(tmp_list[1])
        c_list.append(tmp_list[2])
a_list = str_2_list(a_list)
b_list = str_2_list(b_list)
c_list = str_2_list(c_list)

# Define the dataflow graph
time_steps = 8  # time steps which is the same as the length of the bit-string
input_dim = 2  # number of units in the input layer
hidden_dim = 16  # number of units in the hidden layer
output_dim = 1  # number of units in the output layer
binary_dim = 8
largest_number = pow(2, binary_dim)

tf.reset_default_graph()
# input X and target ouput Y
X = tf.placeholder(tf.float32, [None, time_steps, input_dim], name='x')
Y = tf.placeholder(tf.float32, [None, time_steps], name='y')

# define the RNN cell: can be simple cell, LSTM or GRU
cell = tf.contrib.rnn.BasicRNNCell(num_units=hidden_dim, activation=tf.nn.sigmoid)

# values is a tensor of shape [batch_size, time_steps, hidden_dim]
# last_state is a tensor of shape [batch_size, hidden_dim]
values, last_state = tf.nn.dynamic_rnn(cell, X, dtype=tf.float32)
values = tf.reshape(values, [time_steps, hidden_dim])

# put the values from the RNN through fully-connected layer
W = tf.Variable(tf.random_uniform([hidden_dim, output_dim], minval=-1.0, maxval=1.0), name='W')
b = tf.Variable(tf.zeros([1, output_dim]), name='b')
h = tf.nn.sigmoid(tf.matmul(values, W) + b, name='h')

# minimize loss, using ADAM as weight update rule
h_ = tf.reshape(h, [time_steps])
Y_ = tf.reshape(Y, [time_steps])
loss = tf.reduce_mean(-Y_ * tf.log(h_) - (1 - Y_) * tf.log(1 - h_), name='loss')

train_step = tf.train.AdamOptimizer(0.1).minimize(loss)

# Launch the graph
sess = tf.Session()
sess.run(tf.global_variables_initializer())
start = time.time()
with tf.device("CPU:0"):
    # Number of data for training and testing
    # Please remember the total number is 5000
    num4train = 4000
    num4test = 1000
    epoch = 2
    # train
    for i in range(epoch):
        for j in range(num4train):

            a = np.array(a_list[j], dtype=np.uint8)  # changed
            b = np.array(b_list[j], dtype=np.uint8)  # changed
            c = np.array(c_list[j], dtype=np.uint8)  # changed
            ab = np.c_[a, b]
            x = np.array(ab).reshape([1, binary_dim, 2])
            y = np.array(c).reshape([1, binary_dim])
            if j % 100 == 0:
                print("Epoch: %d Example: %d is running..." % (i, j))
                a = sess.run(loss, feed_dict={X: x, Y: y})
                # print(a)
                # if a < 0.001:
                #     break;
            sess.run(train_step, {X: x, Y: y})
    remain_result = []
    time_result = time.time()-start
    print("10 loops: {:0.2f}ms".format(1000*time_result))
    # print(X.device.endswith("CPU:0"))
    # print(X.device.endswith("GPU:0"))

# Test
# for i in range(num4train + 1, num4train + num4test):
#     a = np.array(a_list[i], dtype=np.uint8)  # changed
#     b = np.array(b_list[i], dtype=np.uint8)  # changed
#     c = np.array(c_list[i], dtype=np.uint8)  # changed
#     ab = np.c_[a, b]
#     x = np.array(ab).reshape([1, binary_dim, 2])
#     y = np.array(c).reshape([1, binary_dim])
#
#     # get predicted value
#     [_probs, _loss] = sess.run([h, loss], {X: x, Y: y})
#     probs = np.array(_probs).reshape([8])
#     prediction = np.array([1 if p >= 0.5 else 0 for p in probs]).reshape([8])
#     # Save the result
#     remain_result.append([prediction, y[0]])
#
#     # calculate error
#     error = np.sum(np.absolute(y - probs))
#
#     # print the prediction, the right y and the error.
#     print("---------------")
#     print(prediction)
#     print(y[0])
#     print(error)
#     print("---------------")
#     print()

sess.close()


Epoch: 0 Example: 0 is running...
Epoch: 0 Example: 100 is running...
Epoch: 0 Example: 200 is running...
Epoch: 0 Example: 300 is running...
Epoch: 0 Example: 400 is running...
Epoch: 0 Example: 500 is running...
Epoch: 0 Example: 600 is running...
Epoch: 0 Example: 700 is running...
Epoch: 0 Example: 800 is running...
Epoch: 0 Example: 900 is running...
Epoch: 0 Example: 1000 is running...
Epoch: 0 Example: 1100 is running...
Epoch: 0 Example: 1200 is running...
Epoch: 0 Example: 1300 is running...
Epoch: 0 Example: 1400 is running...
Epoch: 0 Example: 1500 is running...
Epoch: 0 Example: 1600 is running...
Epoch: 0 Example: 1700 is running...
Epoch: 0 Example: 1800 is running...
Epoch: 0 Example: 1900 is running...
Epoch: 0 Example: 2000 is running...
Epoch: 0 Example: 2100 is running...
Epoch: 0 Example: 2200 is running...
Epoch: 0 Example: 2300 is running...
Epoch: 0 Example: 2400 is running...
Epoch: 0 Example: 2500 is running...
Epoch: 0 Example: 2600 is running...
Epoch: 0 Exam